# 1. 크롤링과 함께 쓰는 mongoDB 예제

### cine21 인물 랭킹 알아내기
  - http://www.cine21.com/rank/person/
  - 조회를 눌렀을 때, Go to Network -> content 의 Request URL/Method 와 Form 데이터 알아내기
    - Request URL: http://www.cine21.com/rank/person/content
    - Request Method: POST
    - Form Data
      - section:actor
      - period_start:2021-10
      - gender:all
      - page:1
    - 하단부 페이지를 누를 때마다, Form Data 의 page 값이 바뀜

In [1]:
import requests as req
import re
import datetime
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
import pymongo

In [3]:
# 몽고DB 서버 연결
username = 'hans'
password = '1234'
conn = pymongo.MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))

In [4]:
# db, collection 연결(없으면 생성)
# python과 연동되었을 때, db와 collection의 경우, document insert 싯점에 생성됨
mongodb = conn.cine21
actor_collection = mongodb.actor_collection

In [5]:
# actor 정보가 있는지 확인하기
actor_list = actor_collection.find()
for actor in actor_list:
    print(actor['actor'])

In [9]:
# request url
cine21_url = 'http://www.cine21.com/rank/person/content'
month = "2021-10"
year = "2020-11"

# post 방식으로 데이터를 가져오기 위한 request 시 필요 정보
conditions = dict()
conditions['section'] = 'actor'
conditions['period_start'] = month
conditions['gender'] = 'all'
conditions['page'] = 1

# post 방식으로 request 요청
response = req.post(cine21_url, data = conditions)

In [11]:
response.url

'http://www.cine21.com/rank/person/content'

In [12]:
response.content

b'\t\t\t<ul class="people_list">\r\n\t\t\t\t<li class="people_li">\r\n\t\t\t\t\t<a href="/db/person/info/?person_id=74826"><img src="https://image.cine21.com/resize/cine21/person/2019/0828/11_54_52__5d65ecfc13ca6[X145,145].jpg" alt="" class="people_thumb" target="_blank" /></a>\r\n\t\t\t\t\t<div class="name"><a href="/db/person/info/?person_id=74826">\xeb\xb0\x95\xec\xa0\x95\xeb\xaf\xbc(1\xed\x8e\xb8)</a></div>\r\n\t\t\t\t\t<ul class="num_info">\r\n\t\t\t\t\t\t<li><span class="tit">\xed\x9d\xa5\xed\x96\x89\xec\xa7\x80\xec\x88\x98</span><strong>161,725</strong></li>\r\n\t\t\t\t\t\t<!--\r\n\t\t\t\t\t\t<li><a href="#" class="btn_graph"><span class="ico"></span><span>\xed\x9d\xa5\xed\x96\x89\xec\x84\xb1\xec\xa0\x81<br />\xea\xb7\xb8\xeb\x9e\x98\xed\x94\x84\xeb\xa1\x9c \xeb\xb3\xb4\xea\xb8\xb0</span></a></li>\r\n\t\t\t\t\t\t-->\r\n\t\t\t\t\t</ul>\r\n\t\t\t\t\t<!-- \xec\x98\x81\xed\x99\x94\xed\x8f\xac\xec\x8a\xa4\xed\x84\xb0\xeb\x8a\x94 \xec\xb5\x9c\xeb\x8c\x80 5\xea\xb0\x9c\xea\xb9\x8c\xec\

In [13]:
dir(response)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [14]:
response.request

<PreparedRequest [POST]>

In [15]:
response.reason

'OK'

In [23]:
dir(response.json)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [ ]:
# 정규 표현식 참고 : https://regexr.com/
# 문자, 숫자를 찾음 : \w
# 특수 기호 () 문자로 인식 : \( \)

# 2. 반복 표현 ?, *, +
# + : 앞 문자가 1번 또는 그 이상 반복되는 패턴
# ? : 앞 문자가 0번 또는 1번 표시되는 패턴(없어도 되고, 한번 있어도 되는 패턴)
# * : 앞 문자가 0번 또는 그이상 반복되는 패턴
import re
test_data = '마동석(17편)'
#re.sub('정규표현식', '변경데이터', test_data)
print(re.sub("\(\w+\)", "", test_data))

In [ ]:
# [실습]
# 한 페이지 내의 배우이름만 가져오기
import re



### 각 배우별 상세 정보를 document에 넣고 싶다.
* 각 배우별 상세 정보를 별도 컬럼으로 만들려했더니, 각 배우별 상세 정보 항목이 다르다!
* 모든 상세 정보 항목을 컬럼으로 만들고, 각 컬럼에 매칭되는 컬럼값을 넣기가 쉽지 않다. 코드도 복잡하고!
* Mongodb는 NoSQL -> 통째로 집어넣자.!

* embedded document
  - document 의 컬럼값으로 document를 넣을 수 있다.

### 흥행지수 뽑기

### 각 배우별 출연 영화를 document에 저장하고 싶다.
  - 출연 영화는 한 개가 될 수도 있고, 여러 개가 될 수도 있음
  - 파이썬은 리스트, mongodb document는 컬럼에 배열(array)로 넣으면 됨 

### insert_one() 로 하나씩 데이터 입력하기 (반복문과 함께 사용하면, 여러 데이터를 넣을 수 있음)

- actor_list: 배우 이름
- actor_details: 배우 상세 정보
- actor_rate: 흥행 지수
- date: 기준월
- movie_list: 출연 영화 리스트!

### collection 삭제하기

In [ ]:
# collection drop
actor_collection.drop()

In [ ]:
# collection 찾기
docs = actor_collection.find()
docs

In [ ]:
for doc in docs:
    print(doc)

### Dictionary 타입으로 만들어서 한번에 insert_many() 로 데이터 입력하기

### Update (컬럼명 변경 예제)

### 컬렉션 객체 이름도 바꿀 수 있겠지요
actor_collection -> actors_info 로 변경